In [1]:
import pyspark

In [2]:
import pyspark.sql

In [3]:
from pyspark.sql.functions import *

In [4]:
spark = pyspark.sql.SparkSession \
        .builder \
        .master('local') \
        .getOrCreate()

In [5]:
dessertMenuDF = spark.read \
                .option('inferSchema', 'true') \
                .option('header', 'true') \
                .format('csv') \
                .load('dessert_menu_w_Schema.csv')

In [6]:
dessertMenuDF.printSchema()

root
 |-- menuId: string (nullable = true)
 |-- name: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- kcal: integer (nullable = true)



In [7]:
import pyspark.sql.types

In [8]:
from pyspark.sql.types import *

In [9]:
mySchema = StructType ( [ StructField('orderId', StringType()), \
                        StructField('menuId', StringType()), \
                        StructField('number', IntegerType())])

In [10]:
dessertOrderDF = spark.readStream \
                    .schema(mySchema) \
                    .format('csv') \
                    .option('path', 'streaming_in/') \
                    .load()

In [11]:
dessertOrderDF.printSchema()

root
 |-- orderId: string (nullable = true)
 |-- menuId: string (nullable = true)
 |-- number: integer (nullable = true)



In [12]:
orderTotal = dessertOrderDF.join(dessertMenuDF, dessertOrderDF.menuId ==dessertMenuDF.menuId) \
                            .groupBy(dessertOrderDF.orderId) \
                            .agg(sum(dessertOrderDF.number * dessertMenuDF.price).alias('orderMoney'))

 # append , complete , update


In [13]:
orderQuery = orderTotal.writeStream \
                        .format('console') \
                        .outputMode('complete') \
                        .start()

In [14]:
type(orderQuery)

pyspark.sql.streaming.StreamingQuery

In [ ]:
orderQuery.awaitTermination()